In [2]:
import torch
import torch.nn as nn

class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)  # Single layer

    def forward(self, x):
        return self.linear(x)

# Initialize
model = LinearRegression(input_dim=1, output_dim=1)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3)  # Input channels=3 (RGB), output=16
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 13 * 13, 10)  # Example output for 10 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 16 * 13 * 13)  # Flatten
        x = self.fc1(x)
        return x

model = CNN()

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)  # out shape: (batch, seq_len, hidden_size)
        out = self.fc(out[:, -1, :])  # Use last timestep
        return out

model = RNN(input_size=10, hidden_size=20, output_size=1)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Example data
X = [[1], [2], [3]]  # Features
y = [[2], [4], [6]]  # Labels
dataset = CustomDataset(X, y)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent

In [ ]:
epochs = 100
for epoch in range(epochs):
    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()  # Clear gradients
        outputs = model(batch_X)  # Forward pass
        loss = criterion(outputs, batch_y)
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
    print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

In [ ]:
with torch.no_grad():  # Disable gradient tracking
    test_X = torch.tensor([[4.0]])  # New data
    pred = model(test_X)
    print(f"Prediction for input 4: {pred.item():.4f}")  # Should be ~8

In [ ]:
model = torchvision.models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 10)  # Replace last layer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
batch_X, batch_y = batch_X.to(device), batch_y.to(device)

In [ ]:
torch.save(model.state_dict(), "model.pth")
model.load_state_dict(torch.load("model.pth"))

In [ ]:
import torchvision
from torchvision import transforms

# Load MNIST dataset
transform = transforms.ToTensor()
train_data = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# Define CNN
class MNIST_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)  # MNIST is 1-channel (grayscale)
        self.fc = nn.Linear(32 * 26 * 26, 10)  # 10 classes

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = x.view(-1, 32 * 26 * 26)
        return self.fc(x)

# Train
model = MNIST_CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

for epoch in range(5):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch}, Loss: {loss.item():.4f}")